In [1]:
!pip install chromadb langchain openai tiktoken

In [2]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [3]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [4]:
!unzip -q new_articles.zip -d new_articles

In [5]:
from dotenv import load_dotenv
import openai
import os
from langchain.llms import OpenAI

load_dotenv()
api_key=os.getenv("openai_api_key")

In [6]:
from langchain.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(api_key=api_key)


/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
loader=DirectoryLoader('/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles',glob="./*.txt")

In [8]:
pip install unstructured #try commenting this and run everything again, you will get error in loader.load. comeback here again and repeat it

Note: you may need to restart the kernel to use updated packages.


In [9]:
docs=loader.load()

In [10]:
docs[0]

Document(metadata={'source': '/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt'}, page_content='Databricks today announced that it has acquired Okera, a data governance platform with a focus on AI. The two companies did not disclose the purchase price. According to Crunchbase, Okera previously raised just under $30 million. Investors include Felicis, Bessemer Venture Partners, Cyber Mentor Fund, ClearSky and Emergent Ventures.\n\nData governance was already a hot topic, but the recent focus on AI has highlighted some of the shortcomings of the previous approach to it, Databricks notes in today’s announcement. “Historically, data governance technologies, regardless of sophistication, rely on enforcing control at some narrow waist layer and require workloads to fit into the ‘walled garden’ at this layer,” the company explains in a blog post. That approach doesn’t work anymore in the age of large lang

In [11]:
docs[0].page_content

'Databricks today announced that it has acquired Okera, a data governance platform with a focus on AI. The two companies did not disclose the purchase price. According to Crunchbase, Okera previously raised just under $30 million. Investors include Felicis, Bessemer Venture Partners, Cyber Mentor Fund, ClearSky and Emergent Ventures.\n\nData governance was already a hot topic, but the recent focus on AI has highlighted some of the shortcomings of the previous approach to it, Databricks notes in today’s announcement. “Historically, data governance technologies, regardless of sophistication, rely on enforcing control at some narrow waist layer and require workloads to fit into the ‘walled garden’ at this layer,” the company explains in a blog post. That approach doesn’t work anymore in the age of large language models (LLMs) because the number of assets is growing too quickly (in part because so much of it is machine-generated) and because the overall AI landscape is changing so quickly,

In [12]:
#now lets split the text
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200
    
)

In [13]:
text=text_splitter.split_documents(docs)

In [14]:
text[0].page_content

'Databricks today announced that it has acquired Okera, a data governance platform with a focus on AI. The two companies did not disclose the purchase price. According to Crunchbase, Okera previously raised just under $30 million. Investors include Felicis, Bessemer Venture Partners, Cyber Mentor Fund, ClearSky and Emergent Ventures.'

In [15]:
len(text)

233

In [16]:
#creating a chroma db

directory="db"



In [17]:
from langchain.vectorstores import Chroma
vector_database=Chroma.from_documents(documents=text, embedding=embeddings, persist_directory=directory)

vector_database.persist

<bound method Chroma.persist of <langchain_community.vectorstores.chroma.Chroma object at 0x7738b5ddbdc0>>

In [18]:
#now since database is created, now lets starting cooking
database=None

In [19]:
database=Chroma(persist_directory=directory, embedding_function=embeddings)

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [20]:
database

In [21]:
#making a retriever for retriving the documnets
retriever=database.as_retriever()
answer=retriever.get_relevant_documents("How much money did Microsoft raise?")

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [22]:
answer

[Document(metadata={'source': '/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”'),
 Document(

In [23]:
#making chain
from langchain.chains import RetrievalQA
llm=OpenAI(api_key=api_key)

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [24]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [25]:
def llm_response(response):
    print(response["result"])
    print("\n\nsources")

    for i in response["source_documents"]:
        print(i.metadata["source"])

In [26]:
query="How much money did Microsoft raise?"

In [27]:
response_of_llm=qa_chain(query)

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [28]:
response_of_llm

{'query': 'How much money did Microsoft raise?',
 'result': " The size of Microsoft's investment is believed to be around $10 billion.",
 'source_documents': [Document(metadata={'source': '/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes

In [29]:
retriever = vector_database.as_retriever(search_kwargs={"k": 2}) #only select first 2 context

In [30]:
docs2 = retriever.get_relevant_documents("How much money did Microsoft raise?")

In [31]:
docs2

[Document(metadata={'source': '/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”'),
 Document(

In [32]:
query="How much money did Microsoft raise?"

In [33]:
response_of_llm=qa_chain(query)

In [34]:
response_of_llm

{'query': 'How much money did Microsoft raise?',
 'result': '\nMicrosoft raised $10 billion.',
 'source_documents': [Document(metadata={'source': '/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}, page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professi

In [35]:
llm_response(response_of_llm)


Microsoft raised $10 billion.


sources
/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-03-checks-the-ai-powered-data-protection-project-incubated-in-area-120-officially-exits-to-google.txt
/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt
/home/saugat/Desktop/GenAI/RAG using genAI/chroma_db/new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt
